## Introduction/Business Problem ##

Vision Zero – to reduce road deaths to almost zero by 2050 - is one goal of the European Union. There are many aspects und factors which can lead to an accidents. One big factor is the weather and the related conditions. 

With available data like weather forecasts, road conditions and the light conditions, it could be possible to warn drivers in order to avoid potential accidents. Since data is real-time available, it would be helpful f there would be a ML-model which could use these parameters in order to predict an accident probaility and severity. Car manufacturer, public health organisations, insurance comanies could be interested in such a model. 

# Data Understanding ##

In this Notebook the suggested data from Coursera was used. This data set includes a huge variety of data. Since the explained objective of this Business Problem is to detect the relation between Weather conditions and severity of accidents, most of the data is not relevant for this use case. It will be evaluated, how big is the impact between the attributes Road conditions, Weather conditionn and light conditions and the accident severity. 
According to the first methods for an initial overview, the data set is 194673 rows and 38 attributs / columns big. In the following lines, the data quality will be evaluated and refined if needed. The target variable severity code is inbalanced, since severity level 1 is 2.5 times more reprensented within the data than severity level2. 

Data will be prepared for modeling --> will be three classification models under evaluation: Decision Tree, kNN, Logistc Regression.


## Data Wrangling ##

In [1]:
#import libraries and read data
import numpy as np
import pandas as pd

df=pd.read_csv("https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv")
df.head()

/Users/meyerste/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [2]:
#get overview with different methods & functions
df.describe()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,INTKEY,SEVERITYCODE.1,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,SDOT_COLCODE,SDOTCOLNUM,SEGLANEKEY,CROSSWALKKEY
count,194673.000000,189339.000000,189339.000000,194673.000000,194673.000000,194673.000000,65070.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,1.149360e+05,194673.000000,1.946730e+05
mean,1.298901,-122.330518,47.619543,108479.364930,141091.456350,141298.811381,37558.450576,1.298901,2.444427,0.037139,0.028391,1.920780,13.867768,7.972521e+06,269.401114,9.782452e+03
std,0.457778,0.029976,0.056157,62649.722558,86634.402737,86986.542110,51745.990273,0.457778,1.345929,0.198150,0.167413,0.631047,6.868755,2.553533e+06,3315.776055,7.226926e+04
min,1.000000,-122.419091,47.495573,1.000000,1001.000000,1001.000000,23807.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.007024e+06,0.000000,0.000000e+00
25%,1.000000,-122.348673,47.575956,54267.000000,70383.000000,70383.000000,28667.000000,1.000000,2.000000,0.000000,0.000000,2.000000,11.000000,6.040015e+06,0.000000,0.000000e+00
50%,1.000000,-122.330224,47.615369,106912.000000,123363.000000,123363.000000,29973.000000,1.000000,2.000000,0.000000,0.000000,2.000000,13.000000,8.023022e+06,0.000000,0.000000e+00
75%,2.000000,-122.311937,47.663664,162272.000000,203319.000000,203459.000000,33973.000000,2.000000,3.000000,0.000000,0.000000,2.000000,14.000000,1.015501e+07,0.000000,0.000000e+00
max,2.000000,-122.238949,47.734142,219547.000000,331454.000000,332954.000000,757580.000000,2.000000,81.000000,6.000000,2.000000,12.000000,69.000000,1.307202e+07,525241.000000,5.239700e+06


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 38 columns):
SEVERITYCODE      194673 non-null int64
X                 189339 non-null float64
Y                 189339 non-null float64
OBJECTID          194673 non-null int64
INCKEY            194673 non-null int64
COLDETKEY         194673 non-null int64
REPORTNO          194673 non-null object
STATUS            194673 non-null object
ADDRTYPE          192747 non-null object
INTKEY            65070 non-null float64
LOCATION          191996 non-null object
EXCEPTRSNCODE     84811 non-null object
EXCEPTRSNDESC     5638 non-null object
SEVERITYCODE.1    194673 non-null int64
SEVERITYDESC      194673 non-null object
COLLISIONTYPE     189769 non-null object
PERSONCOUNT       194673 non-null int64
PEDCOUNT          194673 non-null int64
PEDCYLCOUNT       194673 non-null int64
VEHCOUNT          194673 non-null int64
INCDATE           194673 non-null object
INCDTTM           194673 non-null obje

daytime und uhrzeit umwandeln


In [4]:
df.shape


(194673, 38)

In [5]:
df["WEATHER"].value_counts()

Clear                       111135
Raining                      33145
Overcast                     27714
Unknown                      15091
Snowing                        907
Other                          832
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
Name: WEATHER, dtype: int64

In [6]:
df["ROADCOND"].value_counts()

Dry               124510
Wet                47474
Unknown            15078
Ice                 1209
Snow/Slush          1004
Other                132
Standing Water       115
Sand/Mud/Dirt         75
Oil                   64
Name: ROADCOND, dtype: int64

In [7]:
df["LIGHTCOND"].value_counts()

Daylight                    116137
Dark - Street Lights On      48507
Unknown                      13473
Dusk                          5902
Dawn                          2502
Dark - No Street Lights       1537
Dark - Street Lights Off      1199
Other                          235
Dark - Unknown Lighting         11
Name: LIGHTCOND, dtype: int64

In [8]:
df["SEVERITYCODE"].value_counts()
#target variable is inbalanced. will be balances later with samplesize=58188

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [9]:
df_acc=df[["SEVERITYCODE", "WEATHER", "ROADCOND", "LIGHTCOND"]]

In [10]:
df_acc.head()

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND
0,2,Overcast,Wet,Daylight
1,1,Raining,Wet,Dark - Street Lights On
2,1,Overcast,Dry,Daylight
3,1,Clear,Dry,Daylight
4,2,Raining,Wet,Daylight


In [11]:
# see if there is missing data

missing_data = df_acc.isnull()
missing_data["SEVERITYCODE"].value_counts()



False    194673
Name: SEVERITYCODE, dtype: int64

In [12]:
missing_data["WEATHER"].value_counts()

False    189592
True       5081
Name: WEATHER, dtype: int64

In [13]:
missing_data["ROADCOND"].value_counts()

False    189661
True       5012
Name: ROADCOND, dtype: int64

In [14]:
missing_data["LIGHTCOND"].value_counts()

False    189503
True       5170
Name: LIGHTCOND, dtype: int64

In [15]:
#check if the NaN Cells are mixed up within the coulmns or not
missing_data.loc[missing_data['ROADCOND']]

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND
15,False,True,True,True
36,False,True,True,True
53,False,True,True,True
60,False,True,True,True
75,False,True,True,True
118,False,True,True,True
128,False,True,True,True
132,False,True,True,True
139,False,True,True,True
147,False,True,True,True


In [83]:
#Since most of the missing rows, where there is a missing value, also the other values are NaN, i will drop those cells.



df_new=df_acc.dropna()


In [84]:
#balancing target variable
from sklearn.utils import resample
major= df_new[df_new.SEVERITYCODE==1]
minor=df_new[df_new.SEVERITYCODE==2]
sample= resample(major, replace=False, n_samples=57104, random_state=123)

df_start=pd.concat([sample, minor])

df_start.SEVERITYCODE.value_counts()
df_start.head()


,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND
82770,1,Clear,Dry,Daylight
122946,1,Clear,Dry,Daylight
102968,1,Clear,Dry,Daylight
13906,1,Overcast,Dry,Daylight
123301,1,Raining,Wet,Dark - Street Lights On


In [85]:
df_start["LIGHTCOND"].replace("Daylight", 0, inplace=True)
df_start["LIGHTCOND"].replace("Dark - Street Lights On", 1, inplace=True)
df_start["LIGHTCOND"].replace("Dark - No Street Lights", 2, inplace=True)
df_start["LIGHTCOND"].replace("Dusk", 1, inplace=True)
df_start["LIGHTCOND"].replace("Dawn", 1, inplace=True)
df_start["LIGHTCOND"].replace("Dark - Street Lights Off", 2, inplace=True)
df_start["LIGHTCOND"].replace("Dark - Unknown Lighting", 2, inplace=True)
df_start["LIGHTCOND"].replace("Other",2, inplace=True)
df_start["LIGHTCOND"].replace("Unknown",2, inplace=True)

df_start["WEATHER"].replace("Clear", 0, inplace=True)
df_start["WEATHER"].replace("Raining", 3, inplace=True)
df_start["WEATHER"].replace("Overcast", 1, inplace=True)
df_start["WEATHER"].replace("Other", 2, inplace=True)
df_start["WEATHER"].replace("Unknown", 2, inplace=True)
df_start["WEATHER"].replace("Snowing", 3, inplace=True)
df_start["WEATHER"].replace("Fog/Smog/Smoke", 2, inplace=True)
df_start["WEATHER"].replace("Sleet/Hail/Freezing Rain", 3, inplace=True)
df_start["WEATHER"].replace("Blowing Sand/Dirt", 2, inplace=True)
df_start["WEATHER"].replace("Severe Crosswind", 2, inplace=True)
df_start["WEATHER"].replace("Partly Cloudy", 1, inplace=True)

#Encoding Road Conditions(0 = Dry, 1 = Mushy, 2 = Wet)
df_start["ROADCOND"].replace("Dry", 0, inplace=True)
df_start["ROADCOND"].replace("Wet", 2, inplace=True)
df_start["ROADCOND"].replace("Ice", 2, inplace=True)
df_start["ROADCOND"].replace("Snow/Slush", 1, inplace=True)
df_start["ROADCOND"].replace("Other", 2, inplace=True)
df_start["ROADCOND"].replace("Unknown", 2, inplace=True)
df_start["ROADCOND"].replace("Standing Water", 2, inplace=True)
df_start["ROADCOND"].replace("Sand/Mud/Dirt", 1, inplace=True)
df_start["ROADCOND"].replace("Oil", 2, inplace=True)

In [86]:
#df_start['SEVERITYCODE'].isna().sum()
#df_start['WEATHER'].isna().sum()
#df_start['ROADCOND'].isna().sum()
df_start['LIGHTCOND'].isna().sum()
#df_final.columns

0

In [87]:
df_final=df_start

In [88]:
df_final.columns

Index(['SEVERITYCODE', 'WEATHER', 'ROADCOND', 'LIGHTCOND'], dtype='object')

## Building Model ##

In [89]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [97]:
y= df_final['SEVERITYCODE'].values
x= df_final[['WEATHER', 'ROADCOND', 'LIGHTCOND']].values
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=4)
print(X_train)

[[1 2 0]
 [3 2 2]
 [0 0 0]
 ...
 [3 1 0]
 [2 2 2]
 [0 0 0]]


In [98]:
#starting with KNN, finding the best k
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

array([0.50363525, 0.5097232 , 0.50424842, 0.50915382, 0.5096356 ,
       0.51230729, 0.51528556, 0.51147512, 0.51024877])

In [99]:
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

The best accuracy was with 0.5152855641205326 with k= 7


In [101]:
k=mean_acc.argmax()+1
knn_model = KNeighborsClassifier(n_neighbors=k).fit(x, y)
predKNN=knn_model.predict(X_test)

from sklearn.metrics import classification_report,jaccard_similarity_score,log_loss,f1_score
print(classification_report(y_test,predKNN))
print('\n')
print('Jaccard Similarity Score: ',round(jaccard_similarity_score(y_test,predKNN)*100,2),'%')
print('F1-Score: ',f1_score(y_test,predKNN,average=None))
print('Train Accuracy: ',metrics.accuracy_score(y_train, knn_model.predict(X_train))*100,'%')

             precision    recall  f1-score   support

          1       0.50      0.89      0.64     11457
          2       0.51      0.12      0.20     11375

avg / total       0.51      0.50      0.42     22832



Jaccard Similarity Score:  50.46 %
F1-Score:  [0.64215655 0.19547589]
Train Accuracy:  50.379965835924835 %


ValueError: max_features must be in (0, n_features]

In [ ]:
acctreetree=DecisionTreeClassifier(max_depth=5,criterion='entropy',max_features=2,random_state=0)
acctreetree.fit(X_train,y_train)
pred_acctree=acctree.predict(X_test)

print(classification_report(y_test,pred_acctree))
print('Jaccard Similarity Score : ',round(jaccard_similarity_score(y_test,pred_ltree)*100,2),'%')
print('F1-SCORE : ',f1_score(y_test,pred_acctree,average=None))
print('Train Accuracy: ',metrics.accuracy_score(y_train, acctreetree.predict(X_train))*100,'%')